In [43]:
import os
import json
import numpy as np
from typing import Dict, List, Tuple, Optional
import torch
import open3d as o3d

In [44]:
def format_scan_dict(unformated_dict: Dict, attribute: str) -> Dict:
    # Format raw dictionary of object nodes for all scenes
    scan_list = unformated_dict["scans"]
    formatted_dict = {}
    for scan in scan_list:
        formatted_dict[scan["scan"]] = scan[attribute]
    return formatted_dict


def format_sem_seg_dict(sem_seg_dict: Dict) -> Dict:
    seg_list = sem_seg_dict["segGroups"]
    formatted_dict = {}
    for seg in seg_list:
        formatted_dict[seg["index"]] = seg
    return formatted_dict


def get_dataset_files(scan_dir):
    object_data = json.load(open(os.path.join(scan_dir, "3DSSG", "objects.json")))
    relationship_data = json.load(open(os.path.join(scan_dir, "3DSSG", "relationships.json")))
    objects_dict = format_scan_dict(object_data, "objects")
    relationships_dict = format_scan_dict(relationship_data, "relationships")
    return objects_dict, relationships_dict

def get_semseg_files(scan_dir, scan_id):
    sem_seg_data = json.load(open(os.path.join(scan_dir, "semantic_segmentation_data", scan_id, "semseg.v2.json")))
    sem_seg_dict = format_sem_seg_dict(sem_seg_data)
    return sem_seg_dict

def get_annotated_ply(scan_dir, scan_id):
    ply_path = os.path.join(scan_dir, "semantic_segmentation_data", scan_id, "labels.instances.annotated.v2.ply")
    o3d_ply = o3d.io.read_point_cloud(ply_path)
    return o3d_ply


In [7]:
root = "/mnt/Backup/Dataset/3d_vsg/data"
scan_dir = os.path.join(root, "raw")
scans = json.load(open(os.path.join(scan_dir, "3RScan.json")))
objects_dict, relationships_dict = get_dataset_files(scan_dir)


In [52]:

scan_id = "ddc737b3-765b-241a-9c35-6b7662c04fc9"
sem_seg_dict = get_semseg_files(scan_dir, scan_id)
o3d_ply = get_annotated_ply(scan_dir, scan_id)

pts = sem_seg_dict[1]["segments"]

# select points in o3d_ply with pts as indices
o3d_ply_selected = o3d_ply.select_by_index(pts)
o3d.visualization.draw_geometries([o3d_ply_selected])



In [ ]:
for k, v in sem_seg_dict.items():
    print(k)
    for kk, vv in v.items():
        print(kk, vv)
    break

In [ ]:
for scan in scans:
    for k, v in scan.items():
        print(k, v)
    print()

In [ ]:
for key, value in objects_dict.items():
    print(key, "total objects:", len(value))
    for obj in value:
        new_value = {}
        for k, v in obj.items():
            if k in {"ply_color", "label", "affordances", "id", "global_id", "attributes"}:
                new_value[k] = v
        print(new_value)
    break

In [ ]:
for key, value in relationships_dict.items():
    print(key)
    for rel in value:
        print(rel)
    break

In [ ]:
scan = scans[0]
ref_scan_id = scan["reference"]
rescan_ids = scan["scans"]